# EdTech
Elias Ejo | Jun-16-2023

# Importing the required libraries

In [68]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pytube import YouTube, Playlist

import tensorflow as tf

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Flatten

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import cv2
import mediapipe as mp

import os

# Define folders path:
where source and ouput files are located

In [38]:
ds_folder_path = "Dataset/nptel_ai/"
frame_extr_folder = os.path.join(ds_folder_path, "Frames_Extracted")
feature_extr_folder = "Features_Extracted"
keyframes_folder = "keyframes"
assesments_folder = "Assesments"
output_folder_list = [] 
video_segmentation_sec = 30
frame_extr_per_sec = 8

# Define the keypoint labels
keypoint_labels = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder',
                   'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist',
                   'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']

# Create a folder to store the frames extracted
if not(os.path.exists(frame_extr_folder)):
    os.mkdir(frame_extr_folder) 

# Create segment-features extraction directory if not exists
if not(os.path.exists(feature_extr_folder)):
    os.mkdir(feature_extr_folder) 

# Create segment-keyframes extraction directory if not exists
if not(os.path.exists(keyframes_folder)):
    os.mkdir(keyframes_folder) 

# Create segment assesment result storing directory if not exists
if not(os.path.exists(assesments_folder)):
    os.mkdir(assesments_folder) 

# Download the vidoe from YouTube

In [39]:
# Load playlist dataset file using pandas
df_nptel_ai = pd.read_csv(os.path.join(ds_folder_path, "metadata2.csv"))
df_nptel_ai.head()

,VideoID,PlayListID,Title
0,zWg7U0OEAoE,PLJ5C_6qdAvBFfF7qtFi8Pv_RK8x55jsUQ,How to Learn and Follow the Course


In [6]:
# Construct the playlist URL
playlist_url = f"https://www.youtube.com/watch?v={df_nptel_ai['VideoID'][0]}&list={df_nptel_ai['PlayListID'][0]}"

# Get the video title
video_title = f"{df_nptel_ai['Title'][0]}"

# Construct the video file name
video_file_name = f"{df_nptel_ai['Title'][0]}.mp4"

In [7]:
playlist_items = []

#Accessing the Playlist
playlist = Playlist(playlist_url)

#Checking the number of videos in the playlist
print('Number of videos in playlist: %s' % len(playlist.video_urls))

Number of videos in playlist: 43


In [8]:
#Parsing through the playlist and appending the video urls in play_list
for video_url in playlist.video_urls:
    playlist_items.append(video_url)

In [9]:
playlist_items

['https://www.youtube.com/watch?v=N-DQ8iDlH_U',
 'https://www.youtube.com/watch?v=A_I5XAtkxiY',
 'https://www.youtube.com/watch?v=ueFXTnG8ZC0',
 'https://www.youtube.com/watch?v=668keuD73k4',
 'https://www.youtube.com/watch?v=SJpd7KC18fQ',
 'https://www.youtube.com/watch?v=r6a2WuWPBa0',
 'https://www.youtube.com/watch?v=CdivH5kY_54',
 'https://www.youtube.com/watch?v=feCZo_2Ql3c',
 'https://www.youtube.com/watch?v=-CIto4pV6Zw',
 'https://www.youtube.com/watch?v=OvhQn6HkA_0',
 'https://www.youtube.com/watch?v=zvvUKDfwA6k',
 'https://www.youtube.com/watch?v=9W-VlZFICCY',
 'https://www.youtube.com/watch?v=dN2m8irACSw',
 'https://www.youtube.com/watch?v=PPBynZnJkfM',
 'https://www.youtube.com/watch?v=fWqx01BbeI8',
 'https://www.youtube.com/watch?v=UOt3y50dcb0',
 'https://www.youtube.com/watch?v=ekUGhPsFZZs',
 'https://www.youtube.com/watch?v=LxiFzgQGDBw',
 'https://www.youtube.com/watch?v=M-iDsvrmABw',
 'https://www.youtube.com/watch?v=VpN3zDKjYl0',
 'https://www.youtube.com/watch?v=xD2Zqt

In [12]:
#Looping through the list
for item in playlist_items:
  try:
    yt = YouTube(item, use_oauth=True, allow_oauth_cache=True)
    print('Downloading video: ' + yt.streams[0].title)
 
    if yt.streams[0].title == video_title:
      print("Views :", yt.video_id)
      print("Views :", yt.views)
      print("Description :", yt.description)
      # print("Title :", yt.title)
      print("Duration : {} secs".format(yt.length))
      print("Ratings :", yt.rating)
      #filters out all the files with "mp4" extension
      stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
      stream.download(ds_folder_path)
      # found the video then exist the loop
      break

  except:
    print("Connection Error")

Please open https://www.google.com/device and input code YJKR-PWJH
Views : N-DQ8iDlH_U
Views : 90051
Description : 
Duration : 443 secs
Ratings : None


# Project Task: Week 1

In [26]:
# Saves an extracted frame from a video as an image file.
def save_extracted_frame(frame, output_folder, frame_count, vid_fps):
    # Get current frame timestamp in seconds
    second = (frame_count*5)*vid_fps
    
    if ((second % video_segmentation_sec) == 0) or (output_folder is None):
        # Get the current segmentation number
        seg_no = int(second//video_segmentation_sec) + 1

        output_folder = os.path.join(frame_extr_folder, f"Segment_{seg_no:d}")
        output_folder_list.append(output_folder)
        
        if not(os.path.exists(output_folder)):
            os.mkdir(output_folder)
    
    # Get the current timestamp in milliseconds
    timestamp_ms = (second*1000)

    # Save the frame with the timestamp and frame count
    output_frame_path = os.path.join(output_folder, f"frame_{frame_count:d}_{timestamp_ms:f}_ms.jpg")    
    cv2.imwrite(output_frame_path, frame)

    return output_folder

In [27]:
# Defining a function that extracts frames from a video at uniform time intervals
def extract_frames_uniform_time(video_path):
    
    cap = cv2.VideoCapture(video_path)
    
    # if video is not present, show error
    if not(cap.isOpened()):
        print("Error reading file")

    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_fps = 1.0/vid_fps
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_count   = 0
    output_folder = None

    while frame_count < total_frames:
        # Set the video capture to the current frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count*5)

        # Read the next frame from the video capture object.
        ret, frame = cap.read()
        # - ret: A boolean value indicating whether a frame was successfully read.
        #       - True if a frame was read successfully.
        #       - False if there are no more frames to read or an error occurred.
        # - frame: The next frame from the video, represented as a NumPy array.
        # If frame read is successful, add it to the list
        if ret:
            output_folder = save_extracted_frame(frame, output_folder, frame_count, vid_fps)

        # Increment the current frame by the segment frames
        frame_count   += 1

    cap.release()

In [28]:
# Using a pre-trained VGG16 model for feature extraction. 
# VGG is a convolutional neural network model for image recognition. 
# VGG16 refers to a VGG model with 16 weight layers.
# The input layer takes an image in the size of (224 x 224 x 3), 
# and the output layer is a softmax prediction on 1000 classes. 

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Get the output of the flatten layer
flatten = Flatten()(base_model.output)

# Create a new model with input and output layers
vgg_model = tf.keras.models.Model(inputs=base_model.input, outputs=flatten)

In [29]:
# Extract features from an image using a pretrained VGG16 deep learning model,
def extract_features_from_image(frame_path):
    img = load_img(frame_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Passes the preprocessed image through the VGG16 model to obtain the features.
    features  = vgg_model.predict(x)
    
    return features 

In [30]:
# Save the features to a CSV file
def save_features_df_to_csv(ds_prediction, folder_name):
    # Define the file path to save the CSV file
    csv_file = os.path.join(feature_extr_folder,f"{folder_name}_features.csv")  
    
    ds_prediction.to_csv(csv_file)

    return csv_file

In [31]:
# feature extraction per given segment of the video created and storing the features into a csv file
def extract_features_and_store_to_csv(frame_folder):
    frame_path_list = [] 
    df_prediction = None
    
    for filename in os.listdir(frame_folder):
        # Generate the full path for the frame        
        frame_path = os.path.join(frame_folder, filename)
        # Extract features from the image
        features = extract_features_from_image(frame_path)
    
        if df_prediction is None:
            # first time so instantiate the data frame
            df_prediction = pd.DataFrame(columns=np.arange(features.shape[1]))
        
        # Add a new row to df_prediction using the .loc indexer.
        df_prediction.loc[df_prediction.shape[0]] = features[0]
        frame_path_list.append(frame_path)
    
    df_prediction['Image_path'] = frame_path_list
    
    csv_file = save_features_df_to_csv(df_prediction, os.path.split(frame_folder)[1])
    
    return  csv_file

In [32]:
# Calculate the silhouette score for a given set of data points and cluster labels.
def calculate_silhouette_score(data, labels):
    silhouette_avg = silhouette_score(data, labels, metric='euclidean')
    
    return silhouette_avg

In [33]:
# Specify the range of cluster numbers to evaluate
def get_optimal_number_of_clusters(fea_csv_path):
    
    df_segment_fea = pd.read_csv(fea_csv_path)        
    fea = df_segment_fea.drop(["Unnamed: 0",'Image_path'], axis=1)
    frames = fea.to_numpy()

    # Flatten frames for clustering
    X = frames.reshape(frames.shape[0], -1)

    min_clusters = 2
    max_clusters = 10
    silhouette_scores = []
    
    # Initialize variables to store the maximum silhouette score and the corresponding optimal number of clusters
    max_score = -1
    optimal_clusters = -1

    for n_clusters in range(min_clusters, max_clusters+1):
        # Create the K-means model
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        
        # Fit the model to the data
        kmeans.fit(X)
        
        # Predict the clusters
        labels = kmeans.fit_predict(X)
        
        # Calculate silhouette score
        score = calculate_silhouette_score(X, labels)
        
        silhouette_scores.append(score)

        # Check if the current score is higher than the maximum score so far
        if score > max_score:
            max_score = score
            optimal_clusters = n_clusters
            
    return optimal_clusters

In [69]:
# Function to extract key frames from each cluster in a video segment
def extract_key_frames_to_csv(segment_fea_csv_path, opt_num_clusters, segment):
    
    df_segment_fea = pd.read_csv(segment_fea_csv_path)
    
    fea = df_segment_fea.drop(["Unnamed: 0",'Image_path'], axis=1)
    frames = fea.to_numpy()
    
    # Flatten frames for clustering
    flattened_frames = frames.reshape(frames.shape[0], -1)

    # Apply clustering algorithm
    kmeans = KMeans(n_clusters=opt_num_clusters)    

    # Predict the clusters
    cluster_labels = kmeans.fit_predict(flattened_frames)

    df_segment_fea = df_segment_fea.drop(["Unnamed: 0"], axis=1)
    df_segment_fea['Cluster_Labels'] = cluster_labels

    df_key_frame = pd.DataFrame(columns=["Segment","Cluster", "Image_path"])

    # Iterate over clusters to identify key frames
    for cluster_id in range(kmeans.n_clusters):
        cluster_frames = frames[cluster_labels == cluster_id]
        
        # Calculate cluster centroid
        cluster_centroid = np.mean(cluster_frames, axis=0)
        
        # Calculate distances between frames and centroid
        distances = np.linalg.norm(cluster_frames - cluster_centroid, axis=1)
        
        # Select key frame as the frame with the minimum distance to the centroid
        key_frame_index = np.argmin(distances)
        key_frame_path = df_segment_fea.loc[key_frame_index].Image_path
        df_key_frame.loc[len(df_key_frame.index)] = [segment, cluster_id, key_frame_path]

        # key_frame = cluster_frames[key_frame_index]
    key_frame_csv_file = f"key_frames_of_cluster_segment_{segment}.csv"

    # Save key frame data to CSV file
    df_key_frame.to_csv(os.path.join(keyframes_folder, key_frame_csv_file))

    clustered_frame_csv_file = f"Clustered_frames_segment_{segment}.csv"

    # Save clustred frame data to CSV file
    df_segment_fea.to_csv(os.path.join(keyframes_folder, clustered_frame_csv_file))
    
    return key_frame_csv_file, clustered_frame_csv_file

In [ ]:
#given the video file extract frames from the videos with video segmentaion as defined in the problem
video_path = os.path.join(ds_folder_path, "How to Learn and Follow the Course.mp4")
output_folder_list = []
extract_frames_uniform_time(video_path)

In [ ]:
# Extracting features  from the frames chosen  from the video in segment wise
extracted_features_csv_path_list = []

for output_folder in output_folder_list:
    csv_path = extract_features_and_store_to_csv(output_folder)
    extracted_features_csv_path_list.append(csv_path)

In [41]:
extracted_features_csv_path_list

['Features_Extracted\\Segment_10_features.csv',
 'Features_Extracted\\Segment_11_features.csv',
 'Features_Extracted\\Segment_12_features.csv',
 'Features_Extracted\\Segment_13_features.csv',
 'Features_Extracted\\Segment_14_features.csv',
 'Features_Extracted\\Segment_15_features.csv',
 'Features_Extracted\\Segment_1_features.csv',
 'Features_Extracted\\Segment_2_features.csv',
 'Features_Extracted\\Segment_3_features.csv',
 'Features_Extracted\\Segment_4_features.csv',
 'Features_Extracted\\Segment_5_features.csv',
 'Features_Extracted\\Segment_6_features.csv',
 'Features_Extracted\\Segment_7_features.csv',
 'Features_Extracted\\Segment_8_features.csv',
 'Features_Extracted\\Segment_9_features.csv']

In [42]:
#Key frames extraction process per segment and store the key frames in a csv file
segments_key_frame_csv_path = []
segments_clustered_frame_path =[]

for fea_csv_path in extracted_features_csv_path_list:
    segment_no = fea_csv_path.split('_')[2]
    print('\n \n',fea_csv_path , segment_no)
    
    opt_num_clusters = get_optimal_number_of_clusters(fea_csv_path)
    print('optimul number of cluster extracted')
    
    # Extract key frames from a segment features
    key_frame_path, clustered_frame_path = extract_key_frames_to_csv(fea_csv_path, opt_num_clusters,segment_no)
    print('key frames csv files is created')
    
    segments_key_frame_csv_path.append(key_frame_path)
    segments_clustered_frame_path.append(clustered_frame_path)
    print('Iteration step Completed Sucessfully')


 
 Features_Extracted\Segment_10_features.csv 10
optimul number of cluster extracted
key frames csv files is created
Iteration step Completed Sucessfully

 
 Features_Extracted\Segment_11_features.csv 11
optimul number of cluster extracted
key frames csv files is created
Iteration step Completed Sucessfully

 
 Features_Extracted\Segment_12_features.csv 12
optimul number of cluster extracted
key frames csv files is created
Iteration step Completed Sucessfully

 
 Features_Extracted\Segment_13_features.csv 13
optimul number of cluster extracted
key frames csv files is created
Iteration step Completed Sucessfully

 
 Features_Extracted\Segment_14_features.csv 14
optimul number of cluster extracted
key frames csv files is created
Iteration step Completed Sucessfully

 
 Features_Extracted\Segment_15_features.csv 15
optimul number of cluster extracted
key frames csv files is created
Iteration step Completed Sucessfully

 
 Features_Extracted\Segment_1_features.csv 1
optimul number of clus

In [43]:
# key frames csv path listh in segment wise
segments_key_frame_csv_path

['key_frames_of_cluster_segment_10.csv',
 'key_frames_of_cluster_segment_11.csv',
 'key_frames_of_cluster_segment_12.csv',
 'key_frames_of_cluster_segment_13.csv',
 'key_frames_of_cluster_segment_14.csv',
 'key_frames_of_cluster_segment_15.csv',
 'key_frames_of_cluster_segment_1.csv',
 'key_frames_of_cluster_segment_2.csv',
 'key_frames_of_cluster_segment_3.csv',
 'key_frames_of_cluster_segment_4.csv',
 'key_frames_of_cluster_segment_5.csv',
 'key_frames_of_cluster_segment_6.csv',
 'key_frames_of_cluster_segment_7.csv',
 'key_frames_of_cluster_segment_8.csv',
 'key_frames_of_cluster_segment_9.csv']

In [26]:
# entire frames extracted per segment is stored in this csv files 
# all the frames per segment with their features is stored in this csv files 
segments_clustered_frame_path

['Clustered_frames_segment_10.csv',
 'Clustered_frames_segment_11.csv',
 'Clustered_frames_segment_12.csv',
 'Clustered_frames_segment_13.csv',
 'Clustered_frames_segment_14.csv',
 'Clustered_frames_segment_15.csv',
 'Clustered_frames_segment_1.csv',
 'Clustered_frames_segment_2.csv',
 'Clustered_frames_segment_3.csv',
 'Clustered_frames_segment_4.csv',
 'Clustered_frames_segment_5.csv',
 'Clustered_frames_segment_6.csv',
 'Clustered_frames_segment_7.csv',
 'Clustered_frames_segment_8.csv',
 'Clustered_frames_segment_9.csv']

In [51]:
# df_key_frame = pd.read_csv(os.path.join(keyframes_folder,'key_frames_of_cluster_segment_1.csv'))
df_key_frame = pd.read_csv(os.path.join(keyframes_folder,'Clustered_frames_segment_5.csv'))


# Project Task: Week 2

In [44]:
#detect body keypoints using the MediaPipe
def detect_body_keypoints(image_path):
    #Load the image
    image = cv2.imread(image_path)

    # Convert the image to RGB format (MediaPipe requires RGB images):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a pose detection object
    mp_pose = mp.solutions.pose
    
    # Set a threshold for the minimum confidence score required for human detection
    min_confidence_score = 0.75

    # Initialize the pose detection model
    pose_model = mp_pose.Pose(static_image_mode=True, min_detection_confidence=min_confidence_score)

    # Process the image with pose detection
    results = pose_model.process(image_rgb)

    keypoints  = []

    if results.pose_landmarks is not None:
        # Iterate over the pose landmarks
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            keypoints.append(keypoint_labels[idx])
    else:
        keypoints  = ["No_Key_Points"]
        
    return keypoints

In [45]:
# Perform text detection on an image using the EAST text detection model
def detect_text(image_path):
    # the file path of the pre-trained EAST model
    model_path = 'Model/EAST/frozen_east_text_detection.pb'
    # Load the Frozen EAST Text Detection model:
    net = cv2.dnn.readNet(model_path)

    image = cv2.imread(image_path)
    
    # Preprocess the image by resizing and normalizing
    blob = cv2.dnn.blobFromImage(image, 1.0, (320, 320), (123.68, 116.78, 103.94), swapRB=True, crop=False)

    # Set the input blob for the network
    net.setInput(blob)

    # Forward pass through the network
    scores, geometry = net.forward(['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'])

    # Get the dimensions of the input image
    height, width = image.shape[:2]

    # Set the minimum confidence threshold for text detection
    min_confidence = 0.5

    # Process the output scores and geometry
    boxes = []
    for i in range(scores.shape[2]):
        # Extract scores and geometry data for each detection box
        score = scores[0, 0, i, 0]
        x1 = geometry[0, 0, i, 0] * width
        y1 = geometry[0, 0, i, 1] * height
        x2 = geometry[0, 0, i, 2] * width
        y2 = geometry[0, 0, i, 3] * height
        
        # Filter out weak detections using a confidence threshold
        if score < min_confidence:
            continue

        box_width  = abs(x2 - x1)
        box_height = abs(y2 - y1)
        boxes.append(box_width*box_height)

    total_text_area = sum(boxes)

    if total_text_area >0 :
        text = "Yes"
        img_area = (height*width)
        Percent_of_text = round(total_text_area *100/img_area, 2)    
        text_return_list = [text, Percent_of_text]
    else:
        text = "No"
        Percent_of_text = 0   
        text_return_list = [text, Percent_of_text]

    return text_return_list

In [46]:
# Detect faces in an image using the Haar cascade classifier model
def detect_faces(image_path):
    # the file path of the pre-trained Haar cascade classifier model
    model_path = 'model/Haarcascade/haarcascade_frontalface_default.xml'
    
    # creat a CascadeClassifier object
    face_cascade = cv2.CascadeClassifier(model_path)

    # Load the image
    image = cv2.imread(image_path)
    # grayscale version of the input image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # detect faces in the input image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
    return faces

In [65]:
class HumanDetector:

    def __init__(self, model_path):
        # Load the frozen inference graph
        self.model = tf.compat.v1.GraphDef()
        
        with tf.io.gfile.GFile(model_path, 'rb') as f:
            self.model.ParseFromString(f.read())
       
    def detect_humans(self, image_path):
        # Load and preprocess the image
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(300, 300))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = np.expand_dims(image, axis=0)

        # Get the dimensions of the input image
        height = 300 
        width  = 300

        # Perform inference
        with tf.compat.v1.Session() as sess:
            # input_tensor = tf.compat.v1.placeholder(dtype=tf.float32, shape=(None, None, None, 3), name='image_tensor')
            input_tensor = sess.graph.get_tensor_by_name('image_tensor:0')
           
            tf.import_graph_def(self.model, input_map={'image_tensor': input_tensor})
            output_tensor = sess.graph.get_tensor_by_name('detection_scores:0')
            output_boxes = sess.graph.get_tensor_by_name('detection_boxes:0')
            output_classes = sess.graph.get_tensor_by_name('detection_classes:0')

            scores, boxes, classes = sess.run([output_tensor, output_boxes, output_classes], feed_dict={input_tensor: image})

        # Process the predictions
        scores = scores[0]
        boxes = boxes[0]
        classes = classes[0]

        # Score threshold
        threshold = 0.5
        
        # Area of the input image
        image_area = height*width
        total_box_area = 0
        human_detected = False

        for i in range(len(scores)):
            # Class label 1 represents humans
            if scores[i] > threshold and classes[i] == 1: 
                human_detected = True
                box = boxes[i]
                ymin, xmin, ymax, xmax = box
                box_width = xmax - xmin
                box_height = ymax - ymin
                
                total_box_area += (box_width * box_height)

        human_Percent = round(total_box_area*100/image_area, 2)
        
        return human_detected, human_Percent

In [57]:
# the file path of the pre-trained  frozen TensorFlow model
frozen_model_path = 'Model/Frozen/frozen_inference_graph.pb'

frozen_model = HumanDetector(model_path=frozen_model_path)

# human detetction model and function are defined here
def human_and_interaction_assessment_detection(image_path):
    
    human_detected, human_Percent = frozen_model.detect_humans(image_path)

    if human_detected:
        person = "Yes"
        
        # body keypoints detection in the frame
        body_key_points = detect_body_keypoints(image_path)
        
        # text detection in the frame
        text, text_Percent = detect_text(image_path)
        
        # faces detection in the frame
        faces = detect_faces(image_path)
        
        face = "Yes" if len(faces) > 0 else "No"
        tool = "Board" if human_Percent > 10 else "Slide"

        if (human_Percent > 10):
            assesment = "Interacting with Students" if len(faces) > 0 else "Using Board"
        else:
            assesment = "Using Slide"            
        
        return_list = [image_path, text, text_Percent, person, human_Percent, face, tool, assesment, body_key_points]
    else:
        #text detection in the frame
        text, text_Percent = detect_text(image_path)

        if (text is not None):
            tool = "Slide"
            assesment = "Using Slide"
        else:
            tool = "Null"
            assesment = "Null"
        
        return_list = [image_path, text, text_Percent, "No", 0, "No",  tool, assesment, "No_key_points"]
    
    return return_list

In [70]:
# frame assessment code to be included in the main code 
# assessing the frames based on the objects detected in the image
Assessment_csv_path_list = []
n_segmts = 0 
seg_num_cls = []
seg_cls_key_frame_timestamp = []
  
for key_frame_csv_file in segments_key_frame_csv_path:
    
    # Load segments keyframes .csv file
    df_key_frame = pd.read_csv(os.path.join(keyframes_folder, key_frame_csv_file))

    # video segment number
    seg_number = int(key_frame_csv_file.split('_')[-1].split('.')[0])
    
    n_segmts = n_segmts +1
 
    df_final = pd.DataFrame(columns=['Segment_No', 'Label', 'Image_Path', 'Text', 'Text_%', 'Person', 'Human_%', "Face", 'Body_key_points','Teaching Aid',"Assessment"])
      
    # Counting the number of clusters per segments
    num_cls_per_seg = df_key_frame.shape[0] 
    
    seg_num_cls.append({'seg_{}: {}'.format(seg_number, num_cls_per_seg)})
    
    #iterator over cluster
    cluster_count = 1 
    
    for image_path in df_key_frame['Image_path'].values:
        
        timestamp_ms = image_path.split('_')[-2]
        frame_count  = image_path.split('_')[-3]
        sckfts = f"seg_{seg_number}_cls_{cluster_count}_frame_{frame_count}_timestamp_{timestamp_ms}_ms"
        
        seg_cls_key_frame_timestamp.append(sckfts)

        cluster_count += 1
        
        rtrn_list = human_and_interaction_assessment_detection(image_path)
        
        clust_label = df_key_frame[df_key_frame['Image_path']==image_path].Cluster_Labels.values[0]
        
        rtrn_list.insert(0, clust_label)
        rtrn_list.insert(0, seg_number)
        df_final.loc[df_final.shape[0]] = rtrn_list
    
    asmnt_file_path = os.path.join(assesments_folder,f"segment_{seg_number}_assment.csv")
    Assessment_csv_path_list.append(asmnt_file_path)
    df_final.to_csv(asmnt_file_path)